# Pipeline de entrenamiento y predicción

En este notebook se ejemplifica como ejecutar los scripts para poder:
    
1. Entrenar un modelo para clasificar las zonas de un mapa
2. Usar ese mismo modelo para analizar zonas y tener un estimativo de las zonas 

## Preparando los datos

En este notebook se demostrará el proceso utilizando un recorte de imágenes SPOT6/7 para la ciudad de Concordia, Entre Ríos. Se utilizará la imagen correspondiente al año 2017 para entrenar el modelo, y la del 2018 para evaluarlo.

In [4]:
import os

root_path = os.getcwd()

images_path = os.path.join(root_path, 'data', 'imagenes')
train_tif_path = os.path.join(images_path, 'concordia_2016', 'concordia_small_2016.tif')
test_tif_path = os.path.join(images_path, 'concordia_2018', 'concordia_small_2018.tif')

*Opcional*: Si quiere ejecutar el proceso para las imagenes enteras, descomente las lineas de la siguiente celda y ejecútela.

**Tenga en cuenta que el entrenamiento puede demorar varias horas.**

In [5]:
def download_file(url, out_path):
    if os.path.exists(out_path):
        print("El archivo {} ya existe. Si quiere descargarlo de nuevo, debe borrarlo primero.".format(out_path))
        return

    #!wget -O $test_tif_path $test_remote_tif_url
    r = requests.get(url, stream=True) 
    with open(out_path, "wb") as f: 
        i = 0
        for chunk in r.iter_content(chunk_size=available_mem): 
            # writing one chunk at a time
            if chunk: 
                f.write(chunk)
                clear_output(wait=True)
                print("Escribiendo bloque {} de test, de {} bytes".format(i,available_mem))
                i += 1

In [6]:
# concordia_data_path = '{}/data/conae/centro/concordia'.format(root_path)

# train_product_folder = 'train'
# train_product_image = 'concordia_2016.TIF'
# train_tif_path = '{}/{}/{}'.format(concordia_data_path,train_product_folder,train_product_image)
# train_remote_tif_url = 'https://storage.googleapis.com/dym-ap-siu-habitat-public-images/concordia/IMG_SPOT7_PMS_201601281334024_ORT_C0000000058530_R1C1.TIF'

# !wget --continue -O $train_tif_path $train_remote_tif_url

# test_product_folder = 'test'
# test_product_image = 'concordia_2018.TIF'
# test_tif_path = '{}/{}/{}'.format(concordia_data_path,test_product_folder,test_product_image)
# test_remote_tif_url = 'https://storage.googleapis.com/dym-ap-siu-habitat-public-images/concordia/IMG_SPOT6_PMS_201801071323410_ORT_C0000000058510_R1C1.TIF'

# !wget --continue -O $test_tif_path $test_remote_tif_url

Inicializamos el resto de las variables para ejecutar los scripts, entre ellas la cantidad de hilos paralelos (`n_jobs`) , arboles (`trees`) y profundidad de los mismos (`depth`). Además indicamos las rutas de los scripts, archivos vectoriales etiquetados y rutas de salida.

In [7]:
script_train_path = '{}/script/siu_train.py'.format(root_path)
script_test_path = '{}/script/siu_test.py'.format(root_path)

n_jobs = 4
trees = 107
depth = 1007

train_shp_path = '{}/data/etiquetado/centro/concordia2016_train.shp'.format(root_path)
out_train_model_path = '{}/data/concordia2016_train_rf_t{}_d{}'.format(root_path,trees,depth)
model_file_name = 'concordia2016_train_rf_t{trs}_d{dpt}_rf_t{trs}_d{dpt}.txt'.format(trs=trees,dpt=depth)

test_shp_path = '{}/data/etiquetado/centro/concordia2018_train.shp'.format(root_path)
out_test_results_path = '{}/data/concordia2018_train_rf_t{}_d{}'.format(root_path,trees,depth)

out_train_model_path_full = '{}/model_train_rf_t{}_d{}/{}'.format(out_train_model_path,trees,depth,model_file_name)
out_test_results_path_train = '{}/data/concordia2016_test_rf_t{}_d{}'.format(root_path,trees,depth)

## Entrenamiento del modelo

Entrenamos a modo de ejemplo sobre los datos de la ciudad de concordia del año 2016

In [5]:
# Para ejecutarlo en el notebook, descomentar y usar la siguente linea 
!python3 $script_train_path $train_tif_path $train_shp_path $out_train_model_path $n_jobs $trees $depth --root_path $root_path

# Para ejecutarlo en background independiente del notbook usar la siguente linea
#!python3 $script_train_path $train_tif_path $train_shp_path $out_train_model_path $n_jobs $trees $depth --root_path $root_path >> ./concordia2016_train_rf.log 2>&1

=>    1. Generando features
13:27:00:INFO:409:spfeas.main:
Start date & time --- (Thu Nov 21 13:27:00 2019)

13:27:03:INFO:512:spsplit.get_section_stats:  Processing Fourier transfrom for section 1 of 6 ...
13:28:15:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
13:28:15:INFO:67:spprocess._write_section2file:  Writing section 1 of 6 to file ...
13:28:24:INFO:628:spprocess.run:  Updating status ...
13:28:25:INFO:512:spsplit.get_section_stats:  Processing Fourier transfrom for section 2 of 6 ...
13:29:33:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
13:29:33:INFO:67:spprocess._write_section2file:  Writing section 2 of 6 to file ...
13:29:41:INFO:628:spprocess.run:  Updating status ...
13:29:43:INFO:512:spsplit.get_section_stats:  Processing Fourier transfrom for section 3 of 6 ...
13:30:17:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
13:30:17:INFO:67:spprocess._write_section2file:  Writing section 3 of 6 to file ...
13:30:22:INFO:628:

13:38:26:INFO:512:spsplit.get_section_stats:  Processing Fourier transfrom for section 6 of 6 ...
13:38:26:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
13:38:26:INFO:67:spprocess._write_section2file:  Writing section 6 of 6 to file ...
13:38:26:INFO:628:spprocess.run:  Updating status ...
13:39:26:INFO:512:spsplit.get_section_stats:  Processing Differential Morphological Profiles for section 1 of 6 ...
13:39:26:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
13:39:26:INFO:67:spprocess._write_section2file:  Writing section 1 of 6 to file ...
13:39:31:INFO:628:spprocess.run:  Updating status ...
13:40:20:INFO:512:spsplit.get_section_stats:  Processing Differential Morphological Profiles for section 2 of 6 ...
13:40:21:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
13:40:21:INFO:67:spprocess._write_section2file:  Writing section 2 of 6 to file ...
13:40:26:INFO:628:spprocess.run:  Updating status ...
13:40:49:INFO:512:spsplit.get_section

13:48:03:INFO:628:spprocess.run:  Updating status ...
13:48:04:INFO:512:spsplit.get_section_stats:  Processing Differential Morphological Profiles for section 5 of 6 ...
13:48:04:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
13:48:04:INFO:67:spprocess._write_section2file:  Writing section 5 of 6 to file ...
13:48:05:INFO:512:spsplit.get_section_stats:  Processing Differential Morphological Profiles for section 6 of 6 ...
13:48:05:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
13:48:05:INFO:67:spprocess._write_section2file:  Writing section 6 of 6 to file ...
13:48:05:INFO:628:spprocess.run:  Updating status ...
13:48:05:INFO:512:spsplit.get_section_stats:  Processing Differential Morphological Profiles for section 6 of 6 ...
13:48:05:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
13:48:05:INFO:67:spprocess._write_section2file:  Writing section 6 of 6 to file ...
13:48:06:INFO:628:spprocess.run:  Updating status ...
13:48:07:INFO:512:s

13:56:43:INFO:628:spprocess.run:  Updating status ...
13:56:43:INFO:512:spsplit.get_section_stats:  Processing PanTex for section 5 of 6 ...
13:56:43:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
13:56:43:INFO:67:spprocess._write_section2file:  Writing section 5 of 6 to file ...
13:56:44:INFO:512:spsplit.get_section_stats:  Processing PanTex for section 6 of 6 ...
13:56:44:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
13:56:44:INFO:67:spprocess._write_section2file:  Writing section 6 of 6 to file ...
13:56:44:INFO:628:spprocess.run:  Updating status ...
13:56:44:INFO:512:spsplit.get_section_stats:  Processing PanTex for section 6 of 6 ...
13:56:44:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
13:56:44:INFO:67:spprocess._write_section2file:  Writing section 6 of 6 to file ...
13:56:44:INFO:628:spprocess.run:  Updating status ...
13:56:45:INFO:512:spsplit.get_section_stats:  Processing Local Binary Patterns moments for section 1 of 6 

14:00:30:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:00:30:INFO:67:spprocess._write_section2file:  Writing section 3 of 6 to file ...
14:00:31:INFO:628:spprocess.run:  Updating status ...
14:00:32:INFO:512:spsplit.get_section_stats:  Processing Local Binary Patterns moments for section 4 of 6 ...
14:00:32:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:00:32:INFO:67:spprocess._write_section2file:  Writing section 4 of 6 to file ...
14:00:32:INFO:628:spprocess.run:  Updating status ...
14:00:32:INFO:512:spsplit.get_section_stats:  Processing Local Binary Patterns moments for section 5 of 6 ...
14:00:32:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:00:32:INFO:67:spprocess._write_section2file:  Writing section 5 of 6 to file ...
14:00:32:INFO:512:spsplit.get_section_stats:  Processing Local Binary Patterns moments for section 6 of 6 ...
14:00:32:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:00:32:INFO:67:s

...

14:05:22:INFO:512:spsplit.get_section_stats:  Processing Lacunarity for section 1 of 6 ...
14:05:23:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:05:23:INFO:67:spprocess._write_section2file:  Writing section 1 of 6 to file ...
14:05:25:INFO:628:spprocess.run:  Updating status ...
14:05:25:INFO:512:spsplit.get_section_stats:  Processing Lacunarity for section 2 of 6 ...
14:05:26:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:05:26:INFO:67:spprocess._write_section2file:  Writing section 2 of 6 to file ...
14:05:28:INFO:628:spprocess.run:  Updating status ...
14:05:28:INFO:512:spsplit.get_section_stats:  Processing Lacunarity for section 3 of 6 ...
14:05:29:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:05:29:INFO:67:spprocess._write_section2file:  Writing section 3 of 6 to file ...
14:05:30:INFO:628:spprocess.run:  Updating status ...
14:05:30:INFO:512:spsplit.get_section_stats:  Processing Lacunarity for section 4 of 6 ...
14:

14:05:59:INFO:628:spprocess.run:  Updating status ...
14:06:00:INFO:512:spsplit.get_section_stats:  Processing Normalized Difference Vegetation Index for section 6 of 6 ...
14:06:00:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:06:00:INFO:67:spprocess._write_section2file:  Writing section 6 of 6 to file ...
14:06:00:INFO:628:spprocess.run:  Updating status ...
14:06:01:INFO:512:spsplit.get_section_stats:  Processing Normalized Difference Vegetation Index for section 1 of 6 ...
14:06:01:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:06:01:INFO:67:spprocess._write_section2file:  Writing section 1 of 6 to file ...
14:06:03:INFO:628:spprocess.run:  Updating status ...
14:06:04:INFO:512:spsplit.get_section_stats:  Processing Normalized Difference Vegetation Index for section 2 of 6 ...
14:06:04:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:06:04:INFO:67:spprocess._write_section2file:  Writing section 2 of 6 to file ...
14:06:06:I

14:06:38:INFO:628:spprocess.run:  Updating status ...
14:06:38:INFO:512:spsplit.get_section_stats:  Processing Mean for section 6 of 6 ...
14:06:38:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:06:38:INFO:67:spprocess._write_section2file:  Writing section 6 of 6 to file ...
14:06:38:INFO:628:spprocess.run:  Updating status ...
14:06:39:INFO:512:spsplit.get_section_stats:  Processing Mean for section 1 of 6 ...
14:06:39:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:06:39:INFO:67:spprocess._write_section2file:  Writing section 1 of 6 to file ...
14:06:41:INFO:628:spprocess.run:  Updating status ...
14:06:42:INFO:512:spsplit.get_section_stats:  Processing Mean for section 2 of 6 ...
14:06:42:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:06:42:INFO:67:spprocess._write_section2file:  Writing section 2 of 6 to file ...
14:06:44:INFO:628:spprocess.run:  Updating status ...
14:06:45:INFO:512:spsplit.get_section_stats:  Processing M

14:08:17:INFO:58:sample_raster._sample_parallel:Band 77 of 288 ...
14:08:17:INFO:58:sample_raster._sample_parallel:Band 78 of 288 ...
14:08:18:INFO:58:sample_raster._sample_parallel:Band 79 of 288 ...
14:08:18:INFO:58:sample_raster._sample_parallel:Band 80 of 288 ...
14:08:18:INFO:58:sample_raster._sample_parallel:Band 81 of 288 ...
14:08:18:INFO:58:sample_raster._sample_parallel:Band 82 of 288 ...
14:08:19:INFO:58:sample_raster._sample_parallel:Band 83 of 288 ...
14:08:20:INFO:58:sample_raster._sample_parallel:Band 84 of 288 ...
14:08:20:INFO:58:sample_raster._sample_parallel:Band 85 of 288 ...
14:08:20:INFO:58:sample_raster._sample_parallel:Band 86 of 288 ...
14:08:21:INFO:58:sample_raster._sample_parallel:Band 87 of 288 ...
14:08:22:INFO:58:sample_raster._sample_parallel:Band 88 of 288 ...
14:08:22:INFO:58:sample_raster._sample_parallel:Band 89 of 288 ...
14:08:22:INFO:58:sample_raster._sample_parallel:Band 90 of 288 ...
14:08:22:INFO:58:sample_raster._sample_parallel:Band 91 of 288

14:09:19:INFO:58:sample_raster._sample_parallel:Band 199 of 288 ...
14:09:19:INFO:58:sample_raster._sample_parallel:Band 200 of 288 ...
14:09:21:INFO:58:sample_raster._sample_parallel:Band 203 of 288 ...
14:09:21:INFO:58:sample_raster._sample_parallel:Band 202 of 288 ...
14:09:21:INFO:58:sample_raster._sample_parallel:Band 201 of 288 ...
14:09:21:INFO:58:sample_raster._sample_parallel:Band 204 of 288 ...
14:09:24:INFO:58:sample_raster._sample_parallel:Band 205 of 288 ...
14:09:24:INFO:58:sample_raster._sample_parallel:Band 206 of 288 ...
14:09:24:INFO:58:sample_raster._sample_parallel:Band 207 of 288 ...
14:09:24:INFO:58:sample_raster._sample_parallel:Band 208 of 288 ...
14:09:25:INFO:58:sample_raster._sample_parallel:Band 210 of 288 ...
14:09:25:INFO:58:sample_raster._sample_parallel:Band 209 of 288 ...
14:09:25:INFO:58:sample_raster._sample_parallel:Band 211 of 288 ...
14:09:26:INFO:58:sample_raster._sample_parallel:Band 212 of 288 ...
14:09:27:INFO:58:sample_raster._sample_parallel:

In [6]:
print("Entrenamiento terminado")

Entrenamiento terminado


Podemos analizar la carpeta de salida donde encontramos el resultado de performance y el modelo obtenido

In [7]:
out_folder = '{}/model_train_rf_t{}_d{}/'.format(out_train_model_path,trees,depth)
# !ls -lth $out_folder
os.listdir(out_folder)

['concordia2016_train_rf_t107_d1007_rf_t107_d1007_acc.txt',
 'concordia2016_train_rf_t107_d1007_rf_t107_d1007.txt']

Abriendo el archivo terminado en `_acc.txt` vemos la matriz de confusión y las medidas de performance de entrenamiento

In [8]:
performance_file = '{}{}'.format(out_folder,'concordia2016_train_rf_t107_d1007_rf_t107_d1007_acc.txt')
#!cat $performance_file
with open(performance_file, 'r') as f:
    print(f.read())

Error Matrix

                Observed
                --------
                C     4   C     5   Total   User(%)
                -------   -------   -----   -------
Predicted C004| (54)      0         54      100.00
          C005| 0         (360)     360     100.00
         Total| 54        360       (414)
   Producer(%)| 100.00    100.00            (100.00%)

Samples: 414

Statistics

Overall Accuracy (%): 100.00
Kappa: 1.00
F-beta: 1.00
Hamming loss: 0.00

Class report

             precision    recall  f1-score   support

          4       1.00      1.00      1.00        54
          5       1.00      1.00      1.00       360

avg / total       1.00      1.00      1.00       414



## Predicción usando el modelo aprendido

Usamos el modelo aprendido para analizar la misma zona pero del año 2018


In [9]:
# Para ejecutarlo en el notebook, descomentar y usar la siguente linea 
!python3 $script_test_path $test_tif_path $test_shp_path $out_train_model_path_full $out_test_results_path $n_jobs $trees $depth --root_path $root_path
!python3 $script_test_path $train_tif_path $train_shp_path $out_train_model_path_full $out_test_results_path_train $n_jobs $trees $depth --root_path $root_path

# Para ejecutarlo en background independiente del notbook usar la siguente linea
# Sobre el test set (2018)
#!python3 $script_test_path $test_tif_path $test_shp_path $out_train_model_path_full $out_test_results_path $n_jobs $trees $depth --root_path $root_path >> ./concordia2018_test_rf.log 2>&1
# Sobre el train set (2016)
#!python3 $script_test_path $train_tif_path $train_shp_path $out_train_model_path_full $out_test_results_path_train $n_jobs $trees $depth --root_path $root_path >> ./concordia2016_test_train_rf.log 2>&1

=>    1. Generando features
14:11:57:INFO:409:spfeas.main:
Start date & time --- (Thu Nov 21 14:11:57 2019)

14:11:58:INFO:512:spsplit.get_section_stats:  Processing Fourier transfrom for section 1 of 6 ...
14:12:30:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:12:30:INFO:67:spprocess._write_section2file:  Writing section 1 of 6 to file ...
14:12:35:INFO:628:spprocess.run:  Updating status ...
14:12:35:INFO:512:spsplit.get_section_stats:  Processing Fourier transfrom for section 2 of 6 ...
14:13:08:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:13:08:INFO:67:spprocess._write_section2file:  Writing section 2 of 6 to file ...
14:13:12:INFO:628:spprocess.run:  Updating status ...
14:13:13:INFO:512:spsplit.get_section_stats:  Processing Fourier transfrom for section 3 of 6 ...
14:13:29:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:13:29:INFO:67:spprocess._write_section2file:  Writing section 3 of 6 to file ...
14:13:32:INFO:628:

14:18:16:INFO:512:spsplit.get_section_stats:  Processing Fourier transfrom for section 6 of 6 ...
14:18:16:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:18:16:INFO:67:spprocess._write_section2file:  Writing section 6 of 6 to file ...
14:18:16:INFO:628:spprocess.run:  Updating status ...
14:19:10:INFO:512:spsplit.get_section_stats:  Processing Differential Morphological Profiles for section 1 of 6 ...
14:19:11:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:19:11:INFO:67:spprocess._write_section2file:  Writing section 1 of 6 to file ...
14:19:15:INFO:628:spprocess.run:  Updating status ...
14:20:02:INFO:512:spsplit.get_section_stats:  Processing Differential Morphological Profiles for section 2 of 6 ...
14:20:03:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:20:03:INFO:67:spprocess._write_section2file:  Writing section 2 of 6 to file ...
14:20:07:INFO:628:spprocess.run:  Updating status ...
14:20:29:INFO:512:spsplit.get_section

14:27:59:INFO:628:spprocess.run:  Updating status ...
14:28:00:INFO:512:spsplit.get_section_stats:  Processing Differential Morphological Profiles for section 5 of 6 ...
14:28:00:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:28:00:INFO:67:spprocess._write_section2file:  Writing section 5 of 6 to file ...
14:28:01:INFO:512:spsplit.get_section_stats:  Processing Differential Morphological Profiles for section 6 of 6 ...
14:28:01:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:28:01:INFO:67:spprocess._write_section2file:  Writing section 6 of 6 to file ...
14:28:01:INFO:628:spprocess.run:  Updating status ...
14:28:02:INFO:512:spsplit.get_section_stats:  Processing Differential Morphological Profiles for section 6 of 6 ...
14:28:02:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:28:02:INFO:67:spprocess._write_section2file:  Writing section 6 of 6 to file ...
14:28:02:INFO:628:spprocess.run:  Updating status ...
14:28:03:INFO:512:s

14:35:55:INFO:628:spprocess.run:  Updating status ...
14:35:55:INFO:512:spsplit.get_section_stats:  Processing PanTex for section 5 of 6 ...
14:35:55:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:35:55:INFO:67:spprocess._write_section2file:  Writing section 5 of 6 to file ...
14:35:55:INFO:512:spsplit.get_section_stats:  Processing PanTex for section 6 of 6 ...
14:35:55:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:35:55:INFO:67:spprocess._write_section2file:  Writing section 6 of 6 to file ...
14:35:56:INFO:628:spprocess.run:  Updating status ...
14:35:56:INFO:512:spsplit.get_section_stats:  Processing PanTex for section 6 of 6 ...
14:35:56:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:35:56:INFO:67:spprocess._write_section2file:  Writing section 6 of 6 to file ...
14:35:56:INFO:628:spprocess.run:  Updating status ...
14:35:57:INFO:512:spsplit.get_section_stats:  Processing Local Binary Patterns moments for section 1 of 6 

14:40:06:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:40:06:INFO:67:spprocess._write_section2file:  Writing section 3 of 6 to file ...
14:40:09:INFO:628:spprocess.run:  Updating status ...
14:40:09:INFO:512:spsplit.get_section_stats:  Processing Local Binary Patterns moments for section 4 of 6 ...
14:40:10:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:40:10:INFO:67:spprocess._write_section2file:  Writing section 4 of 6 to file ...
14:40:10:INFO:628:spprocess.run:  Updating status ...
14:40:10:INFO:512:spsplit.get_section_stats:  Processing Local Binary Patterns moments for section 5 of 6 ...
14:40:10:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:40:10:INFO:67:spprocess._write_section2file:  Writing section 5 of 6 to file ...
14:40:10:INFO:512:spsplit.get_section_stats:  Processing Local Binary Patterns moments for section 6 of 6 ...
14:40:10:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
14:40:10:INFO:67:s

...

15:22:29:INFO:512:spsplit.get_section_stats:  Processing Lacunarity for section 3 of 6 ...
15:22:29:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
15:22:29:INFO:67:spprocess._write_section2file:  Writing section 3 of 6 to file ...
15:22:31:INFO:628:spprocess.run:  Updating status ...
15:22:31:INFO:512:spsplit.get_section_stats:  Processing Lacunarity for section 4 of 6 ...
15:22:31:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
15:22:31:INFO:67:spprocess._write_section2file:  Writing section 4 of 6 to file ...
15:22:31:INFO:628:spprocess.run:  Updating status ...
15:22:31:INFO:512:spsplit.get_section_stats:  Processing Lacunarity for section 5 of 6 ...
15:22:31:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
15:22:31:INFO:67:spprocess._write_section2file:  Writing section 5 of 6 to file ...
15:22:31:INFO:512:spsplit.get_section_stats:  Processing Lacunarity for section 6 of 6 ...
15:22:31:INFO:28:spreshape.reshape_feature_list:  Reshapi

15:22:57:INFO:512:spsplit.get_section_stats:  Processing Normalized Difference Vegetation Index for section 1 of 6 ...
15:22:57:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
15:22:57:INFO:67:spprocess._write_section2file:  Writing section 1 of 6 to file ...
15:23:00:INFO:628:spprocess.run:  Updating status ...
15:23:00:INFO:512:spsplit.get_section_stats:  Processing Normalized Difference Vegetation Index for section 2 of 6 ...
15:23:00:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
15:23:00:INFO:67:spprocess._write_section2file:  Writing section 2 of 6 to file ...
15:23:02:INFO:628:spprocess.run:  Updating status ...
15:23:03:INFO:512:spsplit.get_section_stats:  Processing Normalized Difference Vegetation Index for section 3 of 6 ...
15:23:03:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
15:23:03:INFO:67:spprocess._write_section2file:  Writing section 3 of 6 to file ...
15:23:04:INFO:628:spprocess.run:  Updating status ...
15:23:04:I

15:23:30:INFO:628:spprocess.run:  Updating status ...
15:23:31:INFO:512:spsplit.get_section_stats:  Processing Mean for section 1 of 6 ...
15:23:31:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
15:23:31:INFO:67:spprocess._write_section2file:  Writing section 1 of 6 to file ...
15:23:33:INFO:628:spprocess.run:  Updating status ...
15:23:34:INFO:512:spsplit.get_section_stats:  Processing Mean for section 2 of 6 ...
15:23:34:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
15:23:34:INFO:67:spprocess._write_section2file:  Writing section 2 of 6 to file ...
15:23:36:INFO:628:spprocess.run:  Updating status ...
15:23:36:INFO:512:spsplit.get_section_stats:  Processing Mean for section 3 of 6 ...
15:23:36:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
15:23:36:INFO:67:spprocess._write_section2file:  Writing section 3 of 6 to file ...
15:23:37:INFO:628:spprocess.run:  Updating status ...
15:23:38:INFO:512:spsplit.get_section_stats:  Processing M

15:24:50:INFO:58:sample_raster._sample_parallel:Band 87 of 288 ...
15:24:51:INFO:58:sample_raster._sample_parallel:Band 88 of 288 ...
15:24:51:INFO:58:sample_raster._sample_parallel:Band 89 of 288 ...
15:24:51:INFO:58:sample_raster._sample_parallel:Band 90 of 288 ...
15:24:52:INFO:58:sample_raster._sample_parallel:Band 91 of 288 ...
15:24:52:INFO:58:sample_raster._sample_parallel:Band 93 of 288 ...
15:24:52:INFO:58:sample_raster._sample_parallel:Band 92 of 288 ...
15:24:52:INFO:58:sample_raster._sample_parallel:Band 94 of 288 ...
15:24:53:INFO:58:sample_raster._sample_parallel:Band 95 of 288 ...
15:24:53:INFO:58:sample_raster._sample_parallel:Band 96 of 288 ...
15:24:53:INFO:58:sample_raster._sample_parallel:Band 97 of 288 ...
15:24:53:INFO:58:sample_raster._sample_parallel:Band 98 of 288 ...
15:24:54:INFO:58:sample_raster._sample_parallel:Band 99 of 288 ...
15:24:54:INFO:58:sample_raster._sample_parallel:Band 100 of 288 ...
15:24:54:INFO:58:sample_raster._sample_parallel:Band 101 of 2

15:25:44:INFO:58:sample_raster._sample_parallel:Band 208 of 288 ...
15:25:45:INFO:58:sample_raster._sample_parallel:Band 209 of 288 ...
15:25:45:INFO:58:sample_raster._sample_parallel:Band 210 of 288 ...
15:25:45:INFO:58:sample_raster._sample_parallel:Band 211 of 288 ...
15:25:46:INFO:58:sample_raster._sample_parallel:Band 212 of 288 ...
15:25:46:INFO:58:sample_raster._sample_parallel:Band 213 of 288 ...
15:25:46:INFO:58:sample_raster._sample_parallel:Band 214 of 288 ...
15:25:46:INFO:58:sample_raster._sample_parallel:Band 215 of 288 ...
15:25:47:INFO:58:sample_raster._sample_parallel:Band 216 of 288 ...
15:25:47:INFO:58:sample_raster._sample_parallel:Band 217 of 288 ...
15:25:47:INFO:58:sample_raster._sample_parallel:Band 218 of 288 ...
15:25:48:INFO:58:sample_raster._sample_parallel:Band 219 of 288 ...
15:25:48:INFO:58:sample_raster._sample_parallel:Band 220 of 288 ...
15:25:48:INFO:58:sample_raster._sample_parallel:Band 221 of 288 ...
15:25:49:INFO:58:sample_raster._sample_parallel:

In [10]:
print("Evaluacion terminada")

Evaluacion terminada


La performance sobre el año 2018 usado de test es la siguiente

In [11]:
out_folder = '{}/model_test_rf_t{}_d{}/'.format(out_test_results_path,trees,depth)
performance_file = '{}{}'.format(out_folder, 'concordia2018_train_rf_t107_d1007_rf_t107_d1007_acc.txt')
# !cat $performance_file
with open(performance_file, 'r') as f:
    print(f.read())

Error Matrix

                Observed
                --------
                C     2   C     4   C     5   Total   User(%)
                -------   -------   -------   -----   -------
Predicted C002| (0)       0         0         0       0.00
          C004| 122       (0)       0         122     0.00
          C005| 0         0         (823)     823     100.00
         Total| 122       0         823       (945)
   Producer(%)| 0.00      0.00      100.00            (87.09%)

Samples: 945

Statistics

Overall Accuracy (%): 87.09
Kappa: 0.47
Hamming loss: 0.13

Class report

             precision    recall  f1-score   support

          2       0.00      0.00      0.00       122
          4       0.00      0.00      0.00         0
          5       1.00      1.00      1.00       823

avg / total       0.87      0.87      0.87       945



Vemos que en la carpeta `clasificacion_...` dentro de la carpeta de salida, está la imagen obtenida luego de aplicar el modelo

In [12]:
map_file = '{}/clasificacion_test_rf_t{}_d{}/concordia2018_train_rf_t107_d1007_rf_t107_d1007.tif'.format(out_test_results_path, trees, depth)
# !ls $map_file
os.listdir(os.path.dirname(map_file))

['concordia2018_train_rf_t107_d1007_rf_t107_d1007.tif']

## Mover los resultados a una carpeta con formato esperado para post-procesamiento

Movemos los resultados a una estructura de carpetas con el patrón `region/ciudad/año` que esperan los scripts de post-procesamiento 

In [8]:
import subprocess
import glob
import shutil

def prepare_results(*, year, path_salida, region_city, resultados_classifier, imgs_orig, raster_virtual_path):
    # Crear el directorio
    results_full_path = os.path.join(path_salida, region_city, str(year))
    print('Creando directorio {}'.format(results_full_path))
    os.makedirs(os.path.dirname(results_full_path), exist_ok=True)

    # Copiar clasificación a la carpeta
    for tif_file in glob.glob(os.path.join(resultados_classifier, '*.tif')):
        shutil.copy(tif_file,results_full_path)

    # Generar raster virtual .vrt con las imágenes originales
    original_images_full_path = os.path.join(raster_virtual_path, region_city)
    os.makedirs(os.path.join(root_path, original_images_full_path), exist_ok=True)

    cmd = "gdalbuildvrt {dst} {src}".format(
        dst=os.path.join(original_images_full_path, "{}.vrt".format(year)),
        src=os.path.join(imgs_orig, "*.tif"))
    print(cmd)

    process = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE)
    print(process.stdout)

In [9]:
prepare_results(year=2016,
                region_city='centro/concordia',
                path_salida='data/resultados/finales',
                resultados_classifier='data/concordia2016_train_rf_t107_d1007/clasificacion_train_rf_t107_d1007',
                imgs_orig='data/imagenes/concordia_2016',
                raster_virtual_path='data/imagenes_orig')

Creando directorio data/resultados/finales/centro/concordia/2016
gdalbuildvrt data/imagenes_orig/centro/concordia/2016.vrt data/imagenes/concordia_2016/*.tif
b'0...10...20...30...40...50...60...70...80...90...100 - done.\n'


In [10]:
prepare_results(year=2018,
                region_city='centro/concordia',
                path_salida='data/resultados/finales',
                resultados_classifier='data/concordia2018_train_rf_t107_d1007/clasificacion_test_rf_t107_d1007',
                imgs_orig='data/imagenes/concordia_2018',
                raster_virtual_path='data/imagenes_orig')

Creando directorio data/resultados/finales/centro/concordia/2018
gdalbuildvrt data/imagenes_orig/centro/concordia/2018.vrt data/imagenes/concordia_2018/*.tif
b'0...10...20...30...40...50...60...70...80...90...100 - done.\n'
